In [6]:
import multiprocessing
from time import time
from gensim.models import Doc2Vec
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import joblib
import pandas as pd
import numpy as np
from nltk.tokenize import TweetTokenizer
from gensim.models.doc2vec import TaggedDocument

import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/jf/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/jf/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [88]:
all_tweets = joblib.load('./mps_all_tweets.pckle')

all_sentences = []

def extract_sentences(mps_tweets_df):
    mps_tweets_df['author'] = mps_tweets_df['author'].apply(lambda x: x.lower())
    mps_tweets_df['author'] = mps_tweets_df['author'].apply(lambda x: x.split()[0])

    author_contents_mps_tweets_df = mps_tweets_df[['author', 'contents']].copy()

    dict_of_mps = {k: v for k, v in author_contents_mps_tweets_df.groupby('author')['contents']}
    new_dict_of_mps = {}

    for k, v in dict_of_mps.items():
        vals = []
        for val in v:
            tokeniser = TweetTokenizer()
            tokens = tokeniser.tokenize(val)
            filtered_sentence = [w for w in tokens if not w in stopwords.words()]
            vals.append(filtered_sentence)
        new_dict_of_mps[k] = vals
    
    '''
    for k in new_dict_of_mps.keys():
        new_dict_of_mps[k] = [item for sublist in new_dict_of_mps[k] for item in sublist]
    '''

    return new_dict_of_mps

for dict_of_tweets in all_tweets[:-1]:
    all_sentences.append(extract_sentences(pd.DataFrame.from_dict(dict_of_tweets['posts'])))

In [89]:
tagged_sentences = []
for day in all_sentences:
    for mp, sentences in day.items():
        for sentence in sentences:
            tagged_sentences.append(TaggedDocument(sentence, [mp]))

In [90]:
cores = multiprocessing.cpu_count() # Count the number of cores in a computer
cores

d2v_model = Doc2Vec(min_count=2,
                     size=100,
                     sample=0,
                     workers=cores-1,
                     dm=0,  
                     hs=0)

t = time()

d2v_model.build_vocab(tagged_sentences, progress_per=100000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

t = time()

d2v_model.train(tagged_sentences, total_examples=d2v_model.corpus_count, epochs=50, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

d2v_model.init_sims(replace=True)

/usr/local/lib/python3.7/site-packages/gensim/models/doc2vec.py:574: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


Time to build vocab: 0.06 mins
Time to train the model: 7.51 mins


In [91]:
d2v_model.corpus_count

264880

In [92]:
d2v_model.docvecs.most_similar('nigel_farage')

[('nigelmp', 0.5154927968978882),
 ('johnredwood', 0.49262094497680664),
 ('damiancollins', 0.46852439641952515),
 ('colin_j_clark', 0.4410642981529236),
 ('fabianleedsne', 0.43997228145599365),
 ('davidmorrisml', 0.4320324659347534),
 ('gavinshuker', 0.42579740285873413),
 ('cgreenuk', 0.4240879416465759),
 ('kategreensu', 0.4236288070678711),
 ('franciemolloy', 0.4137679636478424)]

In [80]:
tagged_sentences[90]

TaggedDocument(words=['.', '@BrixSoupKitchen', 'vital', 'work', 'across', '#Lambeth', '-', 'pleased', 'see', 'work', 'recognised', '!', 'https://t.co/bnXu3PcRWl'], tags=['chukaumunna'])

In [71]:
d2v_model.docvecs.index2entity

['alanmakmp',
 'albertowenmp',
 'alexsobel',
 'alisonthewliss',
 'andrew4pendle',
 'angusmacneilsnp',
 'annaturley',
 'anneliesedodds',
 'annietrev',
 'borisjohnson',
 'carolynharris24',
 'chrishazzardsf',
 'chrislawsnp',
 'cpjelmore',
 'danirowley',
 'davidlammy',
 'dawnbutlerbrent',
 'edvaizey',
 'geraintdaviesmp',
 'greghands',
 'guyopperman',
 'hannahb4livimp',
 'huddlestonnigel',
 'jeremy_hunt',
 'jimfromoldham',
 'jonathanplaid',
 'jostevenslabour',
 'jwoodcockmp',
 'karlturnermp',
 'kerrymp',
 'lukepollard',
 'marcus4nuneaton',
 'martinjdocherty',
 'marycreaghmp',
 'mikegapes',
 'onnmel',
 'pgibsonsnp',
 'rees4neath',
 'richardbenyonmp',
 'ruthcadbury',
 'sarahchampionmp',
 'sdoughtymp',
 'stellacreasy',
 'theresecoffey',
 'thomasbrake',
 'tobias_ellwood',
 'wesstreeting',
 'willquince',
 'barrysheerman',
 'chionwurah',
 'coyleneil',
 'davidlinden',
 'emmadentcoad',
 'jessphillips',
 'keeleymp',
 'kevinbrennanmp',
 'labourlewis',
 'laylamoran',
 'meaglemp',
 'mike_fabricant',
 '

In [93]:
d2v_model.save('d2v_model.model')